Implementation of basic pytorch tutorial from https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
# Downloading training dataset 
training_data = datasets.FashionMNIST(
    root="data",    # root repoository
    train=True,     # setting training data
    download=True,  # downloading the files
    transform=ToTensor(),   # transform the images to tensor 0..1
)

# Download testing data
test_data = datasets.FashionMNIST(
    root="data",    # root repoository
    train=False,     # setting testing data
    download=True,  # downloading the files
    transform=ToTensor(),   # transform the images to tensor 0..1
)


In [3]:
# Create dataloaders for downloaded datasets
# Dataloaders speed up batch processing

batch_size = 64
train_dataloader = DataLoader(
    dataset=training_data, # Passing the dataset of interest
    batch_size=batch_size   # will load with these batches
)
test_dataloader = DataLoader(
    dataset=test_data, 
    batch_size=batch_size
)

for X, y in train_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

# Numer of batches in dataloader
print(len(train_dataloader))
print(len(test_dataloader))

# Number of images in a dataset (should be approximately equal to number of batcjes * batch size)
print(len(training_data))
print(len(test_data))

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64
938
157
60000
10000


Create model for training. Inheriting form nn.Module
nn.Faltten() by default is responsible for converting all the dimensions starting form 1 to the end into a single dimension.

In [4]:
print(torch.has_mps)
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
device = "cpu"

# MPS - for apple silicon, diesn'tt work for some reason, always outputs 10 perscent accuracy.

print(f"Using device {device}")

# Define model, inheriting form nn.Module
class myNeuralNetwork(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential( # Combines all the operations in it into a single
            nn.Linear(28*28, 512), # linear layer, converts 28*28 values into 512 values
            nn.ReLU(),  # Applies ReLU activation function to the previously outputted tensor.
            nn.Linear(512, 512),    # linear layer, converts 512 values into 512 values
            nn.ReLU(),  # ReLU
            nn.Linear(512, 10)  # linear layer, converts 512 values into 10 values
        )
    def forward(self, x): # applies this function whenever the class instance is called
        # Flatten the image from [1 color, 28, 28] to [28*28], final dimension [batch_size, 28*28]
        # Apply forward
        return self.linear_relu_stack(self.flatten(x))

# Get info
model = myNeuralNetwork().to(device)
print(model)

True
Using device cpu
myNeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Optimize the model, create loss and optimizer instances
https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss
https://pytorch.org/docs/stable/optim.html

In [5]:
loss_fn = nn.CrossEntropyLoss() # Use cross entropy loss,useful for classification problems
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3) # Stochastic gradient descent

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()   # Sets the module in training mode.
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Prediction error loss:
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation --- ??????
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch%100==0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")



Checking model performance

In [6]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    print(size)
    num_batches = len(dataloader)
    model.eval() # Convert to evaluation mode
    test_loss, correct = 0, 0
    with torch.no_grad(): # disable gradient compute
        for X, y in dataloader:
            X, y = X.to(device), y.to(device) # Copy values to device
            pred = model(X) # Compute prediction step
            test_loss += loss_fn(pred, y).item() # add current loss
            # print((pred.argmax(1) == y).type(torch.float).sum().item() )
            correct += (pred.argmax(1) == y).type(torch.float).sum().item() # Argmax gives maximum class from cross entropy
            # We add 1 if classes are equal for every image in the batch
    # print(pred)
    test_loss /= num_batches # Reduce by the number of batches (loss per batch)
    # print(correct)
    correct /= size # correct guesses percentage
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

Train neural network

In [7]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------
loss: 2.289746  [    0/60000]
loss: 2.282587  [ 6400/60000]
loss: 2.259116  [12800/60000]
loss: 2.258056  [19200/60000]
loss: 2.245529  [25600/60000]
loss: 2.202357  [32000/60000]
loss: 2.224488  [38400/60000]
loss: 2.181476  [44800/60000]
loss: 2.179456  [51200/60000]
loss: 2.155030  [57600/60000]
10000
Test Error: 
 Accuracy: 39.9%, Avg loss: 2.144190 

Epoch 2
-------------------
loss: 2.156468  [    0/60000]
loss: 2.143260  [ 6400/60000]
loss: 2.083412  [12800/60000]
loss: 2.097937  [19200/60000]
loss: 2.046090  [25600/60000]
loss: 1.981461  [32000/60000]
loss: 2.010474  [38400/60000]
loss: 1.930524  [44800/60000]
loss: 1.938086  [51200/60000]
loss: 1.858826  [57600/60000]
10000
Test Error: 
 Accuracy: 55.9%, Avg loss: 1.860266 

Epoch 3
-------------------
loss: 1.905449  [    0/60000]
loss: 1.862724  [ 6400/60000]
loss: 1.748428  [12800/60000]
loss: 1.782942  [19200/60000]
loss: 1.672586  [25600/60000]
loss: 1.633474  [32000/60000]
loss: 1.647265  [384